In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna
import json
from sklearn.metrics import mean_absolute_error
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [4]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import optuna

def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_trainnew_a, X_test_new_a, y_train_new_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

X_train_loc_a, X_test_loc_a, y_train_loc_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"], y_train_a], axis=1))
X_train_loc_b, X_test_loc_b, y_train_loc_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"], y_train_b], axis=1))
X_train_loc_c, X_test_loc_c, y_train_loc_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"], y_train_c], axis=1))


X_train_new = pd.concat([X_train_loc_a, X_train_loc_b, X_train_loc_c])
X_test_new = pd.concat([X_test_loc_a, X_test_loc_b, X_test_loc_c])
y_train_new = pd.concat([y_train_loc_a, y_train_loc_b, y_train_loc_c])
y_test = pd.concat([y_test_a, y_test_b, y_test_c])

In [5]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import optuna


def objective(trial, X_train_new_c, y_train_new_c):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 3000),
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        'eval_metric': 'mae'
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train_new_c, y_train_new_c, verbose=300)
    pred = model.predict(X_test_new_c)
    MAE = mean_absolute_error(y_test_c, pred)
    return MAE
    
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train_new_c, y_train_new_c), n_trials=100)

[I 2023-11-11 17:48:21,590] A new study created in memory with name: no-name-ed7c4780-75f4-4f2b-ba25-50afec9b5fb1
[I 2023-11-11 17:48:27,493] Trial 0 finished with value: 54.051558622714424 and parameters: {'n_estimators': 2205, 'learning_rate': 0.00861681614148598, 'max_depth': 7, 'subsample': 0.23165537499254718, 'colsample_bytree': 0.6205674755158046, 'min_child_weight': 5}. Best is trial 0 with value: 54.051558622714424.
[I 2023-11-11 17:48:34,421] Trial 1 finished with value: 58.105648068040736 and parameters: {'n_estimators': 1390, 'learning_rate': 0.001396180272494732, 'max_depth': 7, 'subsample': 0.8253114972245926, 'colsample_bytree': 0.9610189540415709, 'min_child_weight': 5}. Best is trial 0 with value: 54.051558622714424.
[I 2023-11-11 17:48:44,767] Trial 2 finished with value: 54.4916107213996 and parameters: {'n_estimators': 2813, 'learning_rate': 0.0407863909937016, 'max_depth': 9, 'subsample': 0.825747780910301, 'colsample_bytree': 0.3481421842634262, 'min_child_weight'

KeyboardInterrupt: 